In [ ]:
%cd /content
!git clone https://github.com/rica-team/rica-server.git
%cd /content/rica-server

!pip install .[pt]

In [ ]:
from huggingface_hub import login

login()

In [ ]:
import asyncio
import json

from rica.connector import transformers_adapter as tf
from rica.server import RiCA

In [ ]:
# 1. Create a RiCA application instance with a unique package name.
py_app = RiCA("com.example.pyexec", description="An application for executing Python code.")

In [ ]:
# 2. Register tool routes with the application instance.
@py_app.route("/exec", background=False, timeout=5000)
async def _sys_python_exec(input_):
    """
    A tool to execute a single line of Python code.
    input:
    {
        "code": "1+1"
    }
    output:
    {
        "result": "2"
    }
    """
    try:
        code = input_.get("code", "")
        # Using eval is unsafe. For a real application, use a safer execution environment.
        result = eval(code)
        return {"result": str(result)}
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# 3. Create the ReasoningThread.
rt = tf.TransformersReasoningThread(model_name="google/gemma-3-1b-it")

In [ ]:
# 4. Register callbacks.
@rt.token_generated
def _on_token(token: str):
    # This function receives every single token generated by the model.
    # It represents the model's "thinking" process, including tool calls.
    print(token, end="", flush=True)


@rt.trigger
def _on_response(response_payload: any):
    # This function is ONLY called when the model uses the special `rica/response` tool.
    # It's for showing the final, user-facing answer.
    print("\n\n--- USER-FACING RESPONSE ---")
    # Assuming the response payload is a list of content blocks, as per the prompt.
    if isinstance(response_payload, list):
        for item in response_payload:
            if item.get("type") == "text":
                print(item.get("content", ""))
    else:
        print(json.dumps(response_payload, indent=2))
    print("----------------------------\n")

In [ ]:
async def main():
    print("--- Starting RiCA Demo ---")

    # 5. Install the application into the reasoning thread.
    await rt.install(py_app)

    # 6. Start the reasoning thread. It will initialize the model and wait for input.
    rt.run()

    # 7. Insert the initial user prompt. The adapter will format it correctly.
    await rt.insert(
        'Please calculate 123*456 using the tool with package `com.example.pyexec` and route `/exec`. '
        'Think step-by-step about what you need to do, and then use the tool with package `rica` and route `/response` to give me the final answer.'
    )

    # 8. Wait for the generation to complete.
    await rt.wait()

    print("\n\n--- Final Context ---")
    print(rt.context)
    print("---------------------")

    # 9. Clean up resources.
    await rt.destroy()

In [ ]:
asyncio.run(main())